##### STA 141B Data & Web Technologies for Data Analysis

## Lecture 8, 1/27/26, SQL

### Today's topics
- Databases
- Structured Query Language
 
### Datasets

Olympic Games Dataset (taken from Kaggle):
* https://www.kaggle.com/datasets/ramontanoeiro/summer-olympic-medals-1986-2020
* https://www.kaggle.com/datasets/ramontanoeiro/winter-olympic-medals-1924-2018
<br>Thanks to Ramon Tanoeiro for providing these datasets!

We will only need the DB `olympic_games.sqlite` I created based on the kaggle-data. You can find it on Canvas.

### References

* [W3 Schools SQL Tutorial](https://www.w3schools.com/sql/)
* [SQL Cheatsheet](https://www.sqltutorial.org/sql-cheat-sheet/)

## Databases

A _database_ is a collection of data. There are several different models for how to organize data in a database; these are called _database models_. In this context, "model" refers to a design or mental model, not a statistical model.

The _relational model_ organizes data as a collection of tables. Tables have rows (also called _tuples_ or _records_) and columns (also called _attributes_). Most tables have a _key_ column that is unique for each row and _relates_ the table to other tables. The relational model is the most popular database model by far, and the one we'll focus on in this course.

There are also many different software programs for managing databases, called _database management systems_ (DBMS). Each DBMS usually has its own format for storing data on disk, independent of the database model. Some popular DBMSes are:

* [SQLite](https://www.sqlite.org/)
* [MySQL](https://www.mysql.com/)
* [Microsoft SQL Server](https://www.microsoft.com/en-us/sql-server)
* [PostgreSQL](https://www.postgresql.org/)

Why use a database? There are several reasons:

* Your data may already be in a database, so converting to another format is extra work.
* Database operations are highly optimized, so they typically take less time and memory than an equivalent operation in Python.
* Database operations can run on datasets that are too large to fit in memory. Doing this in Python requires special programming strategies.
* Many DBMSes provide built-in version control, multi-user access, and security checks.
* Databases can be updated in real time.

## Structured Query Language

_Structured query language_ (SQL) is a language designed for querying information in relational databases.

A free SQL tutorial is available [here](https://www.w3schools.com/sql/).

### Getting Connected

There are several ways to connect to a database and run SQL queries from Python:

* The built-in __sqlite3__ module, which only supports SQLite.
* The __sqlalchemy__ package, a unified interface for a variety of different SQL database formats (more than just SQLite). See the [tutorial](https://docs.sqlalchemy.org/en/20/tutorial/index.html#unified-tutorial) for more details.

We'll use a SQLite database here, since SQLite is possibly [the most-used database engine in the world](https://sqlite.org/mostdeployed.html). SQLite's popularity is partly due to its reliability, easy setup, and broad range of features.

### Olympic winter games

The Olympic winter games take place in Milano and Cortina from 2/6 to 2/22 __2026__!
You can find more information on [Wikipedia](https://en.wikipedia.org/wiki/2026_Winter_Olympics) and on the [official Webpage](https://www.olympics.com/en/milano-cortina-2026).

We have a database that contains the medal counts / medal tables for all the Olympic Games, both for the winter and the summer variant!

To give you an impression, you can access the medal table for the Winter Olympic Games 2022 in Beijing [here](https://en.wikipedia.org/wiki/2022_Winter_Olympics#Medal_table).

Cortina, located in the northern part of Italy, looks like this (summer/winter):

<img src="../images/cortina_summer.jpg" width=600/>

<img src="../images/cortina_winter.jpg" width=600/>

### Setup


Let's connect to the olympic_games database:

#### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql

#### Connect to DB

To connect to a database, use the module's `connect()` function. This is similar to opening a file; you should close the database when you're done using it.

In [ ]:
# path_db = "./olympic_games.sqlite" # if the DB is in the same folder as this file
path_db = "../data/olympic_games.sqlite"
db = sql.connect(path_db)

In [ ]:
db

#### Execute Commands

#### Database Overview

To execute a SQL query, use the connection's `.execute()` method. This returns a _cursor_, which is a pointer to the results in the database (imagine a finger pointing at the results).

In [ ]:
cur = db.execute("SELECT * FROM winter")

In [ ]:
type(cur)

In [ ]:
cur.fetchall()

In [ ]:
db.commit()

Don't forget to close the database when you're done!

In [ ]:
db.close()

#### DB Metadata

SQLite databases store metadata in a special table called `sqlite_master`. We can use `sqlite_master` to find out the names of the other tables in the database.

In [ ]:
db = sql.connect(path_db)
cur = db.execute("SELECT * FROM sqlite_master")

To get the results from the database, use one of the cursor's fetch methods. The `.fetchall()` method returns all rows in the result.

In [ ]:
cur.fetchall() 

By default, `sqlite3` will return rows as tuples. If you'd rather have the rows as dictionaries indexed by column name, set the `.row_factory` attribute on the database connection.

In [ ]:
db.row_factory = sql.Row 

Now the rows will behave like dictionaries:

In [ ]:
cur = db.execute("SELECT * FROM sqlite_master")
rows = cur.fetchall() 
dr = dict(rows[0])
print(dr)
dr.keys()

In [ ]:
rows

In [ ]:
rows

In [ ]:
df = pd.DataFrame([dict(r) for r in rows])
df

In [ ]:
df_tables = df[df["type"] == "table"] # drop index
df_tables

In [ ]:
cur.execute("PRAGMA table_info(Winter)")
column_info = cur.fetchall()

column_names = {col[0]: col[1] for col in column_info}
column_names

#### `read_sql` in Pandas

To <b>summarize</b>:

We got the data from the database, converted them to a pandas Dataframe and afterwards manipulated the Dataframe. These steps can be done more efficiently by using the `pd.read_sql` function.
The function takes a SQL query and an open database connection as arguments, so you still need to connect to the database first with `sqlite3` or `sqlalchemy`. The result of the query is returned as a data frame.

Thus, we will generally use the `pd.read_sql()` function in __pandas__ to run our SQL queries.

The generic command will look like:
```python
pd.read_sql('''some SQL statement
               here over 
               multiple lines''', db)
```

Where `db` is our connection to the database we established by the command `db = sql.connect(file_path)`.

In [ ]:
pd.read_sql('SELECT * FROM sqlite_master', db)

In [ ]:
db.close()

### Basic Keywords

#### `SELECT`

The `SELECT` command selects rows from a table. Most of your SQL queries will start with `SELECT`. The syntax is:

```sql
SELECT col1, col2, ... FROM my_table;
```

Here `col1`, `col2`, and so on are column names and `my_table` is a table name. You can select all columns with an asterisk  `*`.

SQL is not case-sensitive and ignores whitespace, but the convention is to write SQL keywords in uppercase and column/table names in lowercase. A semicolon `;` marks the end of a SQL query, but this is optional for many tools.

In [ ]:
# db = sql.connect(path_db)

In [ ]:
pd.read_sql('select * from winter', db)

In [ ]:
pd.read_sql('SELECT * FROM winter', db)

In [ ]:
pd.read_sql('SELECT * FROM Summer', db)

In [ ]:
pd.read_sql('SELECT Year, Country_Name, Gold FROM Winter', db)

In [ ]:
pd.read_sql('SELECT Year, Country_Name, Gold FROM Winter;', db) # ; is optional but is preferred

#### `LIMIT`

The `SELECT` command can be extended with many other keywords.

The first of these is `LIMIT`, which limits the number of rows returned. `LIMIT` is the SQL equivalent of Pandas' `.head()` method.

In [ ]:
pd.read_sql('SELECT Year, Country_Name, Gold FROM Winter LIMIT 4', db)

In [ ]:
pd.read_sql('SELECT Year, Country_Name, Gold FROM Winter LIMIT -1', db) # negative numbers will be treated as no limit

#### `DISTINCT`

The `DISTINCT` keyword limits rows to distinct results. `DISTINCT` is the SQL equivalent of Pandas' `.drop_duplicates()` method.

Keep in mind that `DISTINCT` applies to all of the selected columns, not just one column.

In [ ]:
pd.read_sql('SELECT DISTINCT Host_country FROM Winter', db)

In [ ]:
pd.read_sql('SELECT DISTINCT Year, Host_country, Host_city FROM Winter', db) 

In [ ]:
# DISTINCT applies to all selected columns.
# Thus, we get more results here:
pd.read_sql('SELECT DISTINCT Host_country, Host_city FROM Winter', db) 

#### `ORDER BY`

The `ORDER BY` keyword sorts the returned rows. `ORDER BY` is the SQL equivalent of Pandas' `.sort_values()` method.

In [ ]:
pd.read_sql('SELECT Country_Name, Gold, Silver, Bronze FROM Winter ORDER BY Gold', db)

Sorting the values in an ascending matter is not useful here. Thus, we will use the `DESC` keyword.

In [ ]:
pd.read_sql('SELECT Year, Country_Name, Gold, Silver, Bronze FROM Winter ORDER BY Gold DESC LIMIT 10', db)

We can also order the countries with an equal number of gold medals. (Typically we do this by sorting with respect to the number of Silver medals.)

In [ ]:
pd.read_sql('SELECT Country_Name, Gold, Silver, Bronze FROM Winter ORDER BY Gold DESC, Silver DESC, Bronze DESC', db)

Note that we here had to add the `DESC` keyword after each column!

In contrast, if we want to mix different ways of ordering, we only use the keyword `DESC` wherever needed:

In [ ]:
pd.read_sql('SELECT Country_Name, Gold, Silver, Bronze FROM Winter ORDER BY Gold DESC, Country_Name', db)

#### Operators

You can use arithmetic operators `+`, `-`, `*`, `/`, `%` on SQL columns to perform columnwise computations. These are the SQL equivalent of vectorized arithmetic.

In [ ]:
pd.read_sql("SELECT 5*Gold + 2*Silver + Bronze, * FROM Winter", db) # the first two *: multiplication, and the last *: reads all columns.

#### `AS`

You can rename a column with the `AS` keyword. This keyword is especially useful together with SQL arithmetic operators and functions.

In [ ]:
pd.read_sql("SELECT 5*Gold + 2*Silver + Bronze AS medalpoints, * FROM Winter", db)

In [ ]:
pd.read_sql("SELECT 5*Gold + 2*Silver + Bronze AS medalpoints, * FROM Winter ORDER BY medalpoints DESC", db)

### Filters

#### `WHERE`

`WHERE` puts conditions on the rows returned. `WHERE` is the SQL equivalent of subsetting.

You can use `=` to test equality. Other comparison operators, such as `>=`, are also available.

In [ ]:
pd.read_sql('SELECT * FROM Winter WHERE Year = 2018', db)

In [ ]:
# select all countries that got more gold medals than Silver and Bronze medals together
pd.read_sql('SELECT * FROM Winter WHERE Gold > Silver + Bronze', db)

#### `AND` & `OR`

You can use `AND` and `OR` to combine conditions. You can also use parenthesis to indicate the order of operations.

In [ ]:
pd.read_sql('SELECT * FROM Winter WHERE Year >= 2000 AND Gold >= 10', db)

In [ ]:
pd.read_sql('SELECT * FROM Winter WHERE (Year = 2018 OR Year = 2014) AND Gold >= 10', db) # note the brackets here

#### `BETWEEN` / `IN`

The `BETWEEN` ... `AND` keyword is useful for selecting ranges.

In [ ]:
pd.read_sql('SELECT * FROM winter WHERE year >= 2000 AND year <= 2010', db)

In [ ]:
pd.read_sql('SELECT * FROM Winter WHERE Year BETWEEN 2000 AND 2010', db)

You can use `IN` to check whether a value is in a collection of values.

In [ ]:
pd.read_sql('''SELECT DISTINCT Host_country, Host_city, Year FROM Winter 
               WHERE Host_country IN ("Japan", "China", "South Korea", "United States")''', db)

#### `LIKE` / `IS NULL`

SQL's `LIKE` keyword does simple pattern-matching language for strings. This is less powerful than regular expressions, but still useful.

* `%` matches zero or more of any character, similar to regex (re) `.*`
* `_` matches any one character, similar to regex `.`

In other databases (but not SQLite):
* `[]` matches any one of the characters you put inside the brackets, identical to regex `[]`

In [ ]:
pd.read_sql("SELECT DISTINCT Country_Name FROM Winter WHERE Country_Name LIKE 'Austr%'", db)

If we want to look for NULL values, we have to use the `IS NULL` / `IS NOT NULL` keyword(s).

In [ ]:
pd.read_sql("SELECT DISTINCT Country_Name FROM Winter WHERE Country_Name IS NOT NULL", db)

### Clustering

#### Functions & Aggregation

SQL has built-in functions, which vary from one DBMS to another. The SQL cheatsheet lists most of the functions supported by SQLite.

Most SQL functions aggregate data in a column, summarizing that column somehow.

Some of the aggregation functions are:
- COUNT
- SUM
- AVG (=Mean)
- MAX
- MIN

See
[SQL Cheatsheet](https://www.sqltutorial.org/sql-cheat-sheet/)

In [ ]:
pd.read_sql('SELECT MIN(Year), MAX(Year) FROM Winter', db)

In [ ]:
pd.read_sql("SELECT Host_country, COUNT(Country_Name) FROM Winter WHERE Year = 2018", db) 
# 30 countries are listed in our DB for 2018 (well, only 30 countries got medals, I guess)

In [ ]:
pd.read_sql("SELECT Host_country, COUNT(Country_Name) FROM Winter WHERE Year = 2018 AND Gold+Silver+Bronze > 0", db) 
# correct guess!

#### `GROUP BY`

The `GROUP BY` keyword groups rows before they are aggregated. `GROUP BY` is the SQL equivalent of Pandas' `.groupby()` method.

Let's see how many gold medals have been awarded in each year.

In [ ]:
pd.read_sql('SELECT Year, Sum(Gold) FROM Winter GROUP BY Year', db)

In [ ]:
pd.read_sql('''SELECT Country_Name, SUM(Gold) AS totalgold 
               FROM Winter GROUP BY Country_Name 
               ORDER BY totalgold DESC
               LIMIT 10''', db)

<img src="../images/streif.jpg" width=400/>

#### `HAVING`

The `WHERE` keyword puts conditions on the rows returned _before computing any aggregate functions._ So use `WHERE` to remove rows before aggregation.

The `HAVING` keyword puts conditions on the rows returned _after computing any aggregate functions._ So use `HAVING` to remove rows after aggregation. `HAVING` is most useful for conditions on the result of an aggregate.

Note:
Within one table, the keyword `HAVING` must be used after the `GROUP BY` command. In contrast, the `WHERE` statement must be written before the `GROUP BY` statement (within one table).

All countries who won at least 10 Gold medals:

In [ ]:
pd.read_sql('''SELECT Country_Name, SUM(Gold) AS totalgold 
               FROM Winter GROUP BY Country_Name
               HAVING totalgold > 10
               ORDER BY totalgold DESC''', db)

In contrast, this takes only country/years into account where one nation won more than 10 gold medals.

In [ ]:
pd.read_sql('''SELECT Country_Name, SUM(Gold) AS totalgold 
               FROM Winter WHERE Gold > 10 
               GROUP BY Country_Name
               ORDER BY totalgold DESC''', db)

Be careful: The `WHERE` keyword is applied BEFORE clustering, and the `HAVING` keyword is applied AFTER clustering.

In [ ]:
pd.read_sql('SELECT Year, Host_country FROM Winter WHERE Year >= 2000 GROUP BY Host_country', db)

In [ ]:
pd.read_sql('SELECT Year, Host_country FROM Winter GROUP BY Host_country HAVING Year >= 2000', db)

To understand the error, have a look at the following table:

In [ ]:
pd.read_sql('SELECT Year, Host_country FROM Winter GROUP BY Host_country', db)

Since Italy, Canada and the US also had Olympic Winter Games earlier than 2000, they are listed with the earliest date and thus removed by the HAVING keyword.

### Combining Tables

#### Subqueries

You can write one or more `SELECT` queries within another:

First, let's find all nations that have won a gold medal in one single Year and never won a medal in any other Year: 

In [ ]:
pd.read_sql('''SELECT *, COUNT(Country_Name) AS wincount FROM Winter GROUP BY COuntry_Name''', db)

In [ ]:
pd.read_sql('''SELECT w.*
                FROM (SELECT *, COUNT(Country_Name) AS wincount
                      FROM Winter GROUP BY Country_Name) AS w
                WHERE w.wincount = 1 AND w.Gold >= 0
                ORDER BY w.Gold DESC''', db)

Basically, the Unified Team and the OAR are both russian athletes.

To see, which countries hosted the OG more than once, we need a two step procedure.
- First, we create new Dataset where each row represents one version of the Olympic Games (Years + Host_country).
- Second, we group the Dataset of the first step by the Host_country and count the appearances.

In [ ]:
# Our first step would look like:
pd.read_sql('SELECT DISTINCT Year, Host_country FROM Winter', db)

In [ ]:
# Combining the second step with the first step, we have:
pd.read_sql('''SELECT og.Host_country, COUNT(og.Host_country) AS totalhost FROM
                  (SELECT DISTINCT Year, Host_country FROM Winter) AS og
               GROUP BY og.Host_country
               HAVING totalhost > 1''', db)

## END OF FIFTH LECTURE

#### Joins

A `JOIN` combines two tables using a column they have in common. `JOIN` is the SQL equivalent of Pandas' `.join()` method and `pd.merge()` function.

SQLite supports several kinds of joins:
* Left join (`LEFT JOIN`): Keep all rows from the left table, merging rows from the right.
* Inner join (`INNER JOIN`): Keep only rows where there is a match.
* Right join: Keep all rows from the right table, merging rows from the left.
* Full join: Keep all rows from both tables, merging where there are matches.

Let's list all countries that have hosted both the winter and summer OG.

In [ ]:
pd.read_sql('''SELECT DISTINCT wg.Host_Country FROM Winter as wg
                INNER JOIN Summer as sg
                ON wg.Host_Country = sg.Host_Country''', db)

Note that `INNER JOIN` creates the intersection. If we wanted to see all countries that have hosted EITHER the summer or winter OG, we would have used `FULL JOIN`.

In [ ]:
pd.read_sql('''SELECT DISTINCT wg.Host_Country As Winter_host, sg.Host_Country As Summer_host
                FROM Winter as wg
                FULL JOIN Summer as sg
                ON wg.Host_Country = sg.Host_Country''', db)

If we use `LEFT JOIN` we can see which winter hosts have also been summer hosts. But this list does not include summer hosts that have never hosted winter OG.

In [ ]:
pd.read_sql('''SELECT DISTINCT wg.Host_Country As Winter_host, sg.Host_Country As Summer_host
                FROM Winter as wg
                LEFT JOIN Summer as sg
                ON wg.Host_Country = sg.Host_Country''', db)

Let's list all countries that have won gold medals in both the winter and summer olympic games.
For this, we will take multiple steps:
- Create a table of all countries that have won a gold medal in the winter OG.
- Do the same for the summer OG.
- Combine the tables via INNER JOIN.

In [ ]:
# Step1
pd.read_sql('''SELECT Country_Name, SUM(Gold) as totalgold FROM Winter 
               GROUP BY Country_Name HAVING totalgold > 1''', db)

In [ ]:
# Step2
pd.read_sql('''SELECT Country_Name, SUM(Gold) as totalgold FROM Summer 
               GROUP BY Country_Name HAVING totalgold > 1''', db)

In [ ]:
# Combine via INNER JOIN
pd.read_sql('''SELECT wg.Country_Name, wg.totalgold AS wintergold, sg.totalgold AS summergold 
                FROM
                    (SELECT Country_Name, SUM(Gold) as totalgold FROM Winter 
                     GROUP BY Country_Name HAVING totalgold > 1) As wg
                INNER JOIN
                    (SELECT Country_Name, SUM(Gold) as totalgold FROM Summer 
                     GROUP BY Country_Name HAVING totalgold > 1) As sg
                ON wg.Country_Name = sg.Country_Name
                ''', db)

#### VIEWS

This Joins can make a query arbitrarily complicated, but they are very useful. To ease notation, we can use the `VIEW` feature. It allows us to store intermediate data as new tables.

Let's use `VIEW` to solve the task above!

First, we must create a new View with the `CREATE VIEW` keyword:

In [ ]:
db.execute('''CREATE VIEW winter_gold (
                 Country, Gold
              ) AS
              SELECT Country_Name, SUM(Gold) as totalgold FROM Winter 
                 GROUP BY Country_Name HAVING totalgold > 1;''')
db.commit()

Now, we can easily access this table as if it we had loaded it from the DB!

In [ ]:
pd.read_sql("SELECT * FROM winter_gold LIMIT 5", db)

Let's do the same for the Summer Games.

In [ ]:
db.execute('''CREATE VIEW summer_gold (
                 Country, Gold
              ) AS
              SELECT Country_Name, SUM(Gold) as totalgold FROM Summer 
                 GROUP BY Country_Name HAVING totalgold > 1;''')
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM summer_gold LIMIT 5", db)

Now we can use these two tables to solve the foregoing task!

In [ ]:
# Combine via INNER JOIN
pd.read_sql('''SELECT wg.Country, wg.Gold, sg.Gold FROM
                    winter_gold AS wg
                INNER JOIN
                    summer_gold AS sg
                ON wg.Country = sg.Country
                ''', db)

Now, let's see what the 20 most successfull nations in terms of their aggregated gold medals are.
For this, we have to use `FULL JOIN`!

In [ ]:
pd.read_sql('''SELECT wg.Country, wg.Gold, sg.Gold, 
                        wg.Gold + sg.Gold AS olympic_gold 
                FROM
                    winter_gold AS wg
                FULL JOIN
                    summer_gold AS sg
                ON wg.Country = sg.Country
                ORDER BY olympic_gold DESC LIMIT 20
                ''', db)

We can also delete this view after using it. To do so, we have to use the `DROP VIEW` command.

In [ ]:
db.execute('DROP VIEW winter_gold')
db.commit()

In [ ]:
db.execute('DROP VIEW summer_gold')
db.commit()
db.execute('DROP VIEW winter_gold')
db.commit()

We can use the `IF EXISTS` keyword to avoid this error message.

In [ ]:
db.execute('DROP VIEW IF EXISTS winter_gold ')
db.commit()

#### More Examples

Let's find out what was the most succesfull team+year ever in terms of their gold medals.
The solution seems to be very easy:

In [ ]:
pd.read_sql('SELECT * FROM Winter ORDER BY Gold DESC LIMIT 1', db)

However, as we have seen, there are more teams with 14 Gold medals. So how can we solve this without manually adjusting the Limit?

In [ ]:
# Reminder: There are three countries with 14 gold medals
pd.read_sql('SELECT * FROM Winter ORDER BY Gold DESC LIMIT 4', db)

The solution is simple:
- First, we create a table consisting of the maximum amount of gold medals (=14).
- Afterwards, we select every country that achieved this amount of gold medals.

In [ ]:
# Step1
pd.read_sql('SELECT *, MAX(Gold) as maxgold FROM Winter HAVING Gold = maxgold', db)

In [ ]:
# Combine Step1 with Step2
pd.read_sql('''SELECT * FROM
               (SELECT MAX(Gold) as maxgold FROM Winter) AS m
               LEFT JOIN Winter AS w 
               ON m.maxgold = w.Gold''', db)

### DB Operations

#### `INSERT INTO`

In [ ]:
cur = db.execute("""
    INSERT INTO Winter (Year, Host_country, Host_city) 
    VALUES (2022, 'China', 'Beijing')
""")

# Force the database to run ("commit") the query.
# Otherwise the database evaluates queries lazily, so the table will not show up.
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM Winter ORDER BY Year DESC LIMIT 3", db)

Since we haven't specified the medals, we get NULL values, as indicated by the None keyword. To filter for NULL vaules, we have to use the IS NULL or IS NOT NULL phrase.

In [ ]:
pd.read_sql("SELECT DISTINCT Host_country FROM Winter WHERE Country_Name IS NULL", db)

In [ ]:
pd.read_sql("SELECT DISTINCT Host_country FROM Winter WHERE Country_Name IS NOT NULL", db)

#### UPDATE

https://en.wikipedia.org/wiki/2022_Winter_Olympics#Medal_table

In [ ]:
result = db.execute("""UPDATE Winter 
                       SET Country_Name = 'Norway', Gold = 16, Silver = 8, Bronze = 13
                       WHERE Year = 2022""")
db.commit()
pd.read_sql("SELECT * FROM Winter WHERE Year = 2022", db)

Take care: when updating, you typically want to have a WHERE clause. Otherwise, ALL entries of a column will be updated.

#### `DELETE`

In [ ]:
result = db.execute("""
    DELETE FROM Winter WHERE Year = 2022
""")
db.commit()
pd.read_sql("SELECT * FROM Winter WHERE Year = 2022", db)

#### `CREATE`

In general:
CREATE TABLE table_name (
    column1 datatype,
    column2 datatype,
    column3 datatype
)

A list of SQL datatypes can be found [here](https://www.w3schools.com/sql/sql_datatypes.asp).

In [ ]:
result = db.execute("""
    CREATE TABLE new_table (
    first_name varchar(255),
    last_name varchar(255),
    ID int)
""")

# varchar(255): String of length less than 255

db.commit()
pd.read_sql("SELECT * FROM new_table", db)

However, often you don't want to start with an empty Dataset. To ease things, you can create a new table from an existing one via subqueries!

To illustrate this, let's create a new table that contains the total number of medals every Country has achieved.

In [ ]:
pd.read_sql('''SELECT Country_Name, SUM(Gold) AS Gold_tot, SUM(Silver) AS Silver_tot, SUM(Bronze) AS Bronze_tot
               FROM Winter GROUP BY Country_Name 
               ORDER BY Gold_tot DESC, Silver_tot DESC, Bronze_tot DESC, Country_Name ASC''', db)

In [ ]:
result = db.execute("""
    CREATE TABLE All_medals AS
        SELECT Country_Name, SUM(Gold) AS Gold_tot, SUM(Silver) AS Silver_tot, SUM(Bronze) AS Bronze_tot
               FROM Winter GROUP BY Country_Name 
               ORDER BY Gold_tot DESC, Silver_tot DESC, Bronze_tot DESC, Country_Name ASC
""")

db.commit()

In [ ]:
pd.read_sql("SELECT * FROM All_medals LIMIT 10", db)

#### `DROP`

You can completely remove a table with the `DROP` keyword.

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", db)

In [ ]:
db.execute("DROP TABLE new_table;")
db.commit()

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", db)

Gone!

#### Add/Delete Columns

In [ ]:
db.execute("""
    ALTER TABLE all_medals ADD Capital varchar(255)
""")
db.commit()
pd.read_sql("SELECT * FROM all_medals", db)

In [ ]:
db.execute("""
    ALTER TABLE all_medals DROP Capital
""")
db.commit()
pd.read_sql("SELECT * FROM all_medals", db)

### Other stuff

#### SQL Injections

<b>Beware of SQL injections!</b>

You can paste strings into SQL queries, but make sure the string you paste in isn't compromised (e.g., user input)

![Drag Racing](https://imgs.xkcd.com/comics/exploits_of_a_mom_2x.png)

Assume we want to come up with a very easy program that return the number of (total) medals for a country the user specifies.
We can do this as follows:

In [ ]:
db.commit()
df

In [ ]:
country = str(input("Country? "))
df = pd.read_sql("SELECT * FROM all_medals WHERE Country_Name = '" + country + "'", db)
df

In [ ]:
country = "'" + str(input("Country? ")) + "'"
script = "SELECT * FROM all_medals WHERE Country_Name = " + country
print(script)

In [ ]:
cursor = db.cursor()
cur.executescript(script)
db.commit()

In [ ]:
cursor = db.cursor()
country = "'Austria'; DROP TABLE all_medals;--"
script = "SELECT * FROM all_medals WHERE Country_Name = " + country
cursor.executescript(script)
db.commit()

In [ ]:
pd.read_sql('SELECT * FROM all_medals', db)

In [ ]:
# SECURE EXAMPLE: Using parameterized queries
country = str(input("Country? "))
year = int(input('Year? '))
query_secure = "SELECT * FROM Winter WHERE Country_Name = ? AND Year = ?" # Use a placeholder
df_secure = pd.read_sql(query_secure, db, params=[country, year]) # Pass parameters as a list
df_secure

In [ ]:
pd.read_sql('SELECT * FROM Winter', db)

#### Create a database from pandas DF

In [ ]:
df = pd.read_sql('SELECT * FROM Winter', db)

In [ ]:
# Import/Create DB from CSV
csv_file_path = '../data/Winter_Olympic_Medals.csv'
sqlite_db_path = '../data/winter_olympics2.db' 
table_name = 'Winter' # Name of the table to be created in SQLite

conn = sql.connect(sqlite_db_path) # connect to a NEW database!
# Write the DataFrame to a SQLite table
# if_exists options: 'fail', 'replace', 'append'
# index=False prevents Pandas from writing the DataFrame index as a column
df.to_sql(table_name, conn, if_exists='replace', index=False)

In [ ]:
pd.read_sql("SELECT * from sqlite_master", conn)

In [ ]:
conn.close()

#### Create DB from CSV

You can easily store a Pandas df as a sqlite DB by using the `df.to_sql()` command.

Consquently, you can also convert a CSV file to a sqlite DB by using Pandas in between:

In [ ]:
# Import/Create DB from CSV
csv_file_path = '../data/Winter_Olympic_Medals.csv'
sqlite_db_path = '../data/winter_olympics2.db'
table_name = 'Winter' # Name of the table to be created in SQLite

try:
    # Load the CSV data into a Pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    conn = sql.connect(sqlite_db_path)

    # Write the DataFrame to a SQLite table
    # if_exists options: 'fail', 'replace', 'append'
    # index=False prevents Pandas from writing the DataFrame index as a column
    df.to_sql(table_name, conn, if_exists='replace', index=False)

    print(f"Successfully converted '{csv_file_path}' to '{sqlite_db_path}' table '{table_name}'.")

except FileNotFoundError:
    print(f"Error: CSV file not found at '{csv_file_path}'.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the database connection
    if 'conn' in locals() and conn:
        conn.close()

#### Convert DB to CSV/DF

Exporting your DB to a pandas DataFrame is exactly what we have been doing all the time by using the `pd.read_sql()` statement!

In [ ]:
pd.read_sql('''SELECT Country_Name, SUM(Gold) AS Gold_tot, SUM(Silver) AS Silver_tot, SUM(Bronze) AS Bronze_tot
               FROM Winter GROUP BY Country_Name 
               ORDER BY Gold_tot DESC, Silver_tot DESC, Bronze_tot DESC, Country_Name ASC''', db)

The more interesting part is how to convert the db to a CSV file.

In [ ]:
conn = sql.connect(path_db)

df = pd.read_sql('''SELECT Country_Name, SUM(Gold) AS Gold_tot, SUM(Silver) AS Silver_tot, SUM(Bronze) AS Bronze_tot
               FROM Winter GROUP BY Country_Name 
               ORDER BY Gold_tot DESC, Silver_tot DESC, Bronze_tot DESC, Country_Name ASC''', db)

# Export the DataFrame to a CSV file
output_csv_path = '../data/all_olympic_medals.csv' # Desired output CSV filename
df.to_csv(output_csv_path, index=False)

print(f"Data successfully exported to {output_csv_path}")
conn.close()

#### `EXISTS`

The EXISTS operator is a logical operator that checks whether a subquery returns any row.
To negate the EXISTS operator, you can use the NOT EXISTS operator.

Note that if the subquery returns one row with NULL, the result of the EXISTS operator is still true because the result set contains one row with NULL.

For example, if we want to find all countries that have won a medal in Summer Olympic Games, but not in the Winter Olympic Games, we can use the `NOT EXISTS` keyword.

In [ ]:
pd.read_sql('''SELECT DISTINCT Country_Name FROM Summer as sg
               WHERE
                    NOT EXISTS(
                        SELECT 1 FROM Winter WHERE Country_Name = sg.Country_Name
                    )
               ORDER BY Country_Name''', db)

Conversely, to find all countries that have won medals in both version of the Olympic games, we can use the `EXISTS` keyword.

In [ ]:
pd.read_sql('''SELECT DISTINCT Country_Name FROM Summer as sg
               WHERE
                    EXISTS(
                        SELECT 1 FROM Winter WHERE Country_Name = sg.Country_Name
                    )
               ORDER BY Country_Name''', db)

### Other Databases Types

You can use the __sqlalchemy__ package to connect to many different kinds of databases (not only SQLite).

The `sqlalchemy.create_engine()` function creates an _engine_ to connect to a database. The `.raw_connection()` method on the returned engine opens a connection compatible with Pandas.

See the __sqlalchemy__ documentation for more info about [how to write a database URL](https://docs.sqlalchemy.org/en/latest/core/engines.html#database-urls).

In [ ]:
import pandas as pd

In [ ]:
import sqlalchemy

engine = sqlalchemy.create_engine("sqlite:///../data/olympic_games.sqlite")
conn = engine.raw_connection()

In [ ]:
conn

In [ ]:
engine

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master", conn)

In [ ]:
conn.close()

Unlike SQLite, most databases:

* Are not stored in a single file.
* Have a client-server design, where you connect to the database by logging in with a username and password.